In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [3]:
! hdfs dfs -head "/labs/laba01/ml-100k/u.data"

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013
62	257	2	879372434
286	1014	5	879781125
200	222	5	876042340
210	40	3	891035994
224	29	3	888104457
303	785	3	879485318
122	387	5	879270459
194	274	2	879539794
291	1042	4	874834944
234	1184	2	892079237
119	392	4	886176814
167	486	4	892738452
299	144	4	877881320
291	118	2	874833878
308	1	4	887736532
95	546	2	879196566
38	95	5	892430094
102	768	2	883748450
63	277	4	875747401
160	234	5	876861185
50	246	3	877052329
301	98	4	882075827
225	193	4	879539727
290	88	4	880731963
97	194	3	884238860
157	274	4	886890835
181	1081	1	878962623
278	603	5	891295330
276	796	1	874791932
7	32	4	891350932
10	16	4	877888877
284	304	4	885329322
201	979	2	884114233
276	564	3	874791805
287	327	5	875333916
246	201	5	884921594
242	1137	5	879741196
249	241	5	879641194
99	4	5	886519097
178	332	3	882823437
251	100	4	886271884
8

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, LongType

#Создадим схему - user id | item id | rating | timestamp.
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("ts", LongType())
])
schema

StructType(List(StructField(user_id,IntegerType,true),StructField(item_id,IntegerType,true),StructField(rating,IntegerType,true),StructField(ts,LongType,true)))

In [5]:
df = spark.read.csv("/labs/laba01/ml-100k/u.data", sep="\t", schema=schema)
df.show(3)

+-------+-------+------+---------+
|user_id|item_id|rating|       ts|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
+-------+-------+------+---------+
only showing top 3 rows



In [6]:
hist_all_raw = sorted(df.groupBy('rating').agg({'item_id': 'count'}).collect())
hist_film_raw = sorted(df.where(df.item_id == 318).groupBy('rating').agg({'item_id': 'count'}).collect())
hist_all_raw

[Row(rating=1, count(item_id)=6110),
 Row(rating=2, count(item_id)=11370),
 Row(rating=3, count(item_id)=27145),
 Row(rating=4, count(item_id)=34174),
 Row(rating=5, count(item_id)=21201)]

In [7]:
hist_all = [rating.asDict().get('count(item_id)') for rating in hist_all_raw]
hist_film = [rating.asDict().get('count(item_id)') for rating in hist_film_raw] 

result = {'hist_all': hist_all, 'hist_film':hist_film}
result

{'hist_all': [6110, 11370, 27145, 34174, 21201],
 'hist_film': [4, 6, 23, 79, 186]}

In [8]:
import json
with open("lab01.json", 'w') as f:
    json.dump(result, f)

In [9]:
!pwd

/data/home/igor.dmitriev


In [10]:
!ls

HelloSpark.ipynb  lab01.json  lab1.ipynb


In [11]:
with open("lab01.json", 'r') as f:
    print(f.read())

{"hist_all": [6110, 11370, 27145, 34174, 21201], "hist_film": [4, 6, 23, 79, 186]}


In [13]:
spark.stop()